# --- PARAMS (edit these) ------------------------------------------------------
catalog_name = "dev_sihan"  # your catalog
managed_root = "abfss://uc@mystorageaccount.dfs.core.windows.net/dev_sihan"  
# ^ root for the catalog's managed location (should be empty/new)

schemas = ["bronze", "silver", "gold"]  # schemas to create
schema_subdirs = True  # put each schema under a subfolder of managed_root

# Optional: set an owner and grant access to groups
set_owner_principal = "account users"  # could be a group or a user; or None to skip
grant_groups = ["data-engineers", "analytics-readers"]  # Azure Entra ID groups synced to UC

# What to grant:
grant_create_in_catalog_to = ["data-engineers"]  # can create schemas/tables
grant_usage_in_catalog_to = grant_groups         # can USE CATALOG
grant_usage_in_schemas_to = grant_groups         # can USE SCHEMA, SELECT, etc.
# -----------------------------------------------------------------------------


In [ ]:

import os

# Read catalog and managed root from environment variables
catalog = os.getenv("catalog")   # set in bundle.yml variables
managed_root = os.getenv("managed_root")  # also from bundle.yml
diisengineers = os.getenv("diisengineers")  # set in bundle.yml variables

# Optional helper to safely quote identifiers
def qident(name: str) -> str:
    return f"`{name.replace('`','``')}`"

# 1) Create catalog with managed location (idempotent)
spark.sql(f"""
CREATE CATALOG IF NOT EXISTS {qident(catalog)};
""")
# no storage location specified, so it defaults to Unity Catalog root

# 2) Set owner : make the group the OWNER of the catalog ----------------
spark.sql(f"ALTER CATALOG {qident(catalog)} OWNER TO `{diisengineers}`")
print(f"✅ {diisengineers} set as OWNER of catalog {catalog}")

# 3) Conditional extra grant if catalog == main_dev
if catalog == "main_dev":
    diisanalysts = os.getenv("diisanalysts", "diisanalysts")  # default if not set
    spark.sql(f"GRANT USE CATALOG ON CATALOG {qident(catalog)} TO {qident(diisanalysts)}")
    print(f"✅ Granted USE CATALOG on {catalog} to {diisanalysts}")

print("✅ Catalog and grants ensured for job clusters.")

print("✅ Catalog, schemas, and grants ensured for job clusters.")
